<a href="https://colab.research.google.com/github/MJawadJafri/quiz_game-using-datasturucture-and-filehandling-in-c-/blob/main/Supervised_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Default Parameters

In [1]:
filename = "bank-full.csv"
target_variable_name = "y"

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=d08587e8428296b2f5cb546f2e65402e76b0a8201b5f8b722cc0803a42065f47
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


# Load Dataset

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
data = spark.read.csv(filename, header=True, inferSchema=True, sep=';')
data.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown|  single|  unknown|     no|      1|     no|  no|unknown|  5|  may

# Length of the data

In [4]:
data.count()

45211

# Describe data

In [5]:
data.describe().toPandas()

,summary,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,count,45211,45211,45211,45211,45211,45211,45211,45211,45211,45211,45211,45211,45211,45211,45211,45211,45211
1,mean,40.93621021432837,None,None,None,None,1362.2720576850766,None,None,None,15.80641879188693,None,258.1630797814691,2.763840658246887,40.19782796222158,0.5803233726305546,None,None
2,stddev,10.618762040975408,None,None,None,None,3044.7658291685243,None,None,None,8.322476153044596,None,257.52781226517095,3.098020883279184,100.12874599059813,2.3034410449312204,None,None
3,min,18,admin.,divorced,primary,no,-8019,no,no,cellular,1,apr,0,1,-1,0,failure,no
4,max,95,unknown,single,unknown,yes,102127,yes,yes,unknown,31,sep,4918,63,871,275,unknown,yes


# Check Data types of each column

In [6]:
data.groupby('marital').count().show()

+--------+-----+
| marital|count|
+--------+-----+
|divorced| 5207|
| married|27214|
|  single|12790|
+--------+-----+



In [7]:
data.dtypes

[('age', 'int'),
 ('job', 'string'),
 ('marital', 'string'),
 ('education', 'string'),
 ('default', 'string'),
 ('balance', 'int'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('contact', 'string'),
 ('day', 'int'),
 ('month', 'string'),
 ('duration', 'int'),
 ('campaign', 'int'),
 ('pdays', 'int'),
 ('previous', 'int'),
 ('poutcome', 'string'),
 ('y', 'string')]

In [8]:
data.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [9]:
from pyspark.sql.functions import * 
data.groupBy(target_variable_name).agg({'balance':'avg', 'age': 'avg'}).show()

+---+------------------+------------------+
|  y|      avg(balance)|          avg(age)|
+---+------------------+------------------+
| no|1303.7149691899203| 40.83898602274435|
|yes|1804.2679145396105|41.670069956513515|
+---+------------------+------------------+



# Cardinality Check

In [10]:
from pyspark.sql.functions import approxCountDistinct, countDistinct

"""
Note: approxCountDistinct and countDistinct can be used interchangeably. Only difference is the computation time. 

"approxCountDistinct" is useful for large datasets 
"countDistinct" for small and medium datasets.

"""

def cardinality_calculation(df, cut_off=1):
    cardinality = df.select(*[approxCountDistinct(c).alias(c) for c in df.columns])
    
    ## convert to pandas for efficient calculations
    final_cardinality_df = cardinality.toPandas().transpose()
    final_cardinality_df.reset_index(inplace=True) 
    final_cardinality_df.rename(columns={0:'Cardinality'}, inplace=True) 
    
    #select variables with cardinality of 1
    vars_selected = final_cardinality_df['index'][final_cardinality_df['Cardinality'] <= cut_off] 
    
    return final_cardinality_df, vars_selected

cardinality_df, cardinality_vars_selected = cardinality_calculation(data)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/functions.py:2610: FutureWarning: Deprecated in 2.1, use approx_count_distinct instead.
  warnings.warn("Deprecated in 2.1, use approx_count_distinct instead.", FutureWarning)


In [11]:
cardinality_df

,index,Cardinality
0,age,76
1,job,11
2,marital,3
3,education,4
4,default,2
5,balance,7375
6,housing,2
7,loan,2
8,contact,3
9,day,32


In [12]:
cardinality_vars_selected

Series([], Name: index, dtype: object)

# Missing value check

In [13]:
#missing values check
from pyspark.sql.functions import count, when, isnan, col

# miss_percentage is set to 80% as discussed in the book
def missing_calculation(df, miss_percentage=0.80):
    
    #checks for both NaN and null values
    missing = df.select(*[count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])
    length_df = df.count()
    ## convert to pandas for efficient calculations
    final_missing_df = missing.toPandas().transpose()
    final_missing_df.reset_index(inplace=True) 
    final_missing_df.rename(columns={0:'missing_count'}, inplace=True) 
    final_missing_df['missing_percentage'] = final_missing_df['missing_count']/length_df
    
    #select variables with cardinality of 1
    vars_selected = final_missing_df['index'][final_missing_df['missing_percentage'] >= miss_percentage] 
    
    return final_missing_df, vars_selected

In [14]:
missing_df, missing_vars_selected = missing_calculation(data)

In [15]:
missing_df

,index,missing_count,missing_percentage
0,age,0,0.0
1,job,0,0.0
2,marital,0,0.0
3,education,0,0.0
4,default,0,0.0
5,balance,0,0.0
6,housing,0,0.0
7,loan,0,0.0
8,contact,0,0.0
9,day,0,0.0


In [16]:
missing_vars_selected

Series([], Name: index, dtype: object)

# Identify variable types

In [17]:
def variable_type(df):
    
    vars_list = df.dtypes
    char_vars = []
    num_vars = []
    for i in vars_list:
        if i[1] in ('string'):
            char_vars.append(i[0])
        else:
            num_vars.append(i[0])
    
    return char_vars, num_vars

In [18]:
char_vars, num_vars = variable_type(data)

In [19]:
char_vars

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'poutcome',
 'y']

In [20]:
num_vars

['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

In [21]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

def category_to_index(df, char_vars):
    
    char_df = df.select(char_vars)
    indexers = [StringIndexer(inputCol=c, outputCol=c+"_index", handleInvalid="keep") for c in char_df.columns]
    pipeline = Pipeline(stages=indexers)
    char_labels = pipeline.fit(char_df)
    df = char_labels.transform(df)
    return df, char_labels

In [22]:
data, char_labels = category_to_index(data, char_vars)

In [23]:
data.dtypes

[('age', 'int'),
 ('job', 'string'),
 ('marital', 'string'),
 ('education', 'string'),
 ('default', 'string'),
 ('balance', 'int'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('contact', 'string'),
 ('day', 'int'),
 ('month', 'string'),
 ('duration', 'int'),
 ('campaign', 'int'),
 ('pdays', 'int'),
 ('previous', 'int'),
 ('poutcome', 'string'),
 ('y', 'string'),
 ('job_index', 'double'),
 ('marital_index', 'double'),
 ('education_index', 'double'),
 ('default_index', 'double'),
 ('housing_index', 'double'),
 ('loan_index', 'double'),
 ('contact_index', 'double'),
 ('month_index', 'double'),
 ('poutcome_index', 'double'),
 ('y_index', 'double')]

In [24]:
data = data.select([c for c in data.columns if c not in char_vars])

In [25]:
def rename_columns(df, char_vars):
    mapping = dict(zip([i + '_index' for i in char_vars], char_vars))
    df = df.select([col(c).alias(mapping.get(c, c)) for c in df.columns])
    return df

In [26]:
data = rename_columns(data, char_vars)

In [27]:
data.dtypes

[('age', 'int'),
 ('balance', 'int'),
 ('day', 'int'),
 ('duration', 'int'),
 ('campaign', 'int'),
 ('pdays', 'int'),
 ('previous', 'int'),
 ('job', 'double'),
 ('marital', 'double'),
 ('education', 'double'),
 ('default', 'double'),
 ('housing', 'double'),
 ('loan', 'double'),
 ('contact', 'double'),
 ('month', 'double'),
 ('poutcome', 'double'),
 ('y', 'double')]

In [28]:
data.groupBy('y').count().show() 

+---+-----+
|  y|count|
+---+-----+
|0.0|39922|
|1.0| 5289|
+---+-----+



In [29]:
linear_df = data.select(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous'])

In [30]:
linear_df

DataFrame[age: int, balance: int, day: int, duration: int, campaign: int, pdays: int, previous: int]

In [31]:
target_variable_name = 'balance'

# Assemble input vectors

In [32]:
from pyspark.ml.feature import VectorAssembler

#assemble individual columns to one column - 'features'
def assemble_vectors(df, features_list, target_variable_name):
    stages = []
    #assemble vectors
    assembler = VectorAssembler(inputCols=features_list, outputCol='features')
    stages = [assembler]
    #select all the columns + target + newly created 'features' column
    selectedCols = [target_variable_name, 'features'] + features_list
    #use pipeline to process sequentially
    pipeline = Pipeline(stages=stages)
    #assembler model
    assembleModel = pipeline.fit(df)
    #apply assembler model on data
    df = assembleModel.transform(df).select(selectedCols)

    return df

In [33]:
#exclude target variable and select all other feature vectors
features_list = linear_df.columns
#features_list = char_vars #this option is used only for ChiSqselector
features_list.remove(target_variable_name)

In [34]:
features_list

['age', 'day', 'duration', 'campaign', 'pdays', 'previous']

In [35]:
# apply the function on our dataframe
df = assemble_vectors(linear_df, features_list, target_variable_name)

In [36]:
df.show()

+-------+--------------------+---+---+--------+--------+-----+--------+
|balance|            features|age|day|duration|campaign|pdays|previous|
+-------+--------------------+---+---+--------+--------+-----+--------+
|   2143|[58.0,5.0,261.0,1...| 58|  5|     261|       1|   -1|       0|
|     29|[44.0,5.0,151.0,1...| 44|  5|     151|       1|   -1|       0|
|      2|[33.0,5.0,76.0,1....| 33|  5|      76|       1|   -1|       0|
|   1506|[47.0,5.0,92.0,1....| 47|  5|      92|       1|   -1|       0|
|      1|[33.0,5.0,198.0,1...| 33|  5|     198|       1|   -1|       0|
|    231|[35.0,5.0,139.0,1...| 35|  5|     139|       1|   -1|       0|
|    447|[28.0,5.0,217.0,1...| 28|  5|     217|       1|   -1|       0|
|      2|[42.0,5.0,380.0,1...| 42|  5|     380|       1|   -1|       0|
|    121|[58.0,5.0,50.0,1....| 58|  5|      50|       1|   -1|       0|
|    593|[43.0,5.0,55.0,1....| 43|  5|      55|       1|   -1|       0|
|    270|[41.0,5.0,222.0,1...| 41|  5|     222|       1|   -1|  

# Linear Regression

In [37]:
from pyspark.ml.regression import LinearRegression
reg = LinearRegression(featuresCol='features', labelCol='balance')
reg_model = reg.fit(df) # fit model

In [38]:
import pandas as pd
for k, v in df.schema["features"].metadata["ml_attr"]["attrs"].items():
    features_df = pd.DataFrame(v)

# print coefficient and intercept
print(reg_model.coefficients, reg_model.intercept)

features_df['coefficients'] = reg_model.coefficients

[28.08397290892997,3.3055463619496286,0.24882841970901756,-14.142676297161454,-0.08248810233032043,23.462992800762525] 124.92130092818479


In [39]:
features_df

,idx,name,coefficients
0,0,age,28.083973
1,1,day,3.305546
2,2,duration,0.248828
3,3,campaign,-14.142676
4,4,pdays,-0.082488
5,5,previous,23.462993


In [40]:
#prediction result
pred_result = reg_model.transform(df)

In [41]:
pred_result

DataFrame[balance: int, features: vector, age: int, day: int, duration: int, campaign: int, pdays: int, previous: int, prediction: double]

In [42]:
reg_model.summary.r2

0.010568116511551984

In [43]:
features_list

['age', 'day', 'duration', 'campaign', 'pdays', 'previous']

# Variance Inflation factor

In [44]:
def vif_calculator(df, features_list):
    vif_list = []
    for i in features_list:
        temp_features_list = features_list.copy()
        temp_features_list.remove(i)
        temp_target = i
        assembler = VectorAssembler(inputCols=temp_features_list, outputCol='features')
        temp_df = assembler.transform(df)
        reg = LinearRegression(featuresCol='features', labelCol=i)
        reg_model = reg.fit(temp_df) # fit model
        temp_vif = 1/(1 - reg_model.summary.r2)
        vif_list.append(temp_vif)
    return vif_list

In [45]:
features_df['vif'] = vif_calculator(linear_df, features_list)

In [46]:
features_df

,idx,name,coefficients,vif
0,0,age,28.083973,1.000917
1,1,day,3.305546,1.034350
2,2,duration,0.248828,1.007627
3,3,campaign,-14.142676,1.039907
4,4,pdays,-0.082488,1.276182
5,5,previous,23.462993,1.261321


# Logistic Regression

In [47]:
target_variable_name = "y"
logistic_df = data.select(['age', 'balance', 'day', 'duration', 'campaign', 
'pdays', 'previous', 'y'])
#exclude target variable and select all other feature vectors
features_list = logistic_df.columns
#features_list = char_vars #this option is used only for ChiSqselector
features_list.remove(target_variable_name)
# apply the function on our dataframe
df = assemble_vectors(logistic_df, features_list, target_variable_name)

In [48]:
import numpy as np
from pyspark.ml.classification import LogisticRegression
binary_clf = LogisticRegression(featuresCol='features', labelCol='y', 
family='binomial')

In [50]:
multinomial_clf = LogisticRegression(featuresCol='features', labelCol='y', family='multinomial')
# binary_clf_model = binary_clf.fit(df) # fit binary model
multinomial_clf_model = multinomial_clf.fit(df) # fit multinomial model
np.set_printoptions(precision=3, suppress=True) #model coefficients for binary model
# print(binary_clf_model.coefficients) #model coefficients for multinomial model
np.set_printoptions(precision=4, suppress=True)
print(multinomial_clf_model.coefficientMatrix)
#print(binary_clf_model.intercept) #model intercept for binary model #model intercept for multinomial model
print(multinomial_clf_model.interceptVector)

DenseMatrix([[ 0.0161,  0.    ,  0.0207,  0.0038, -0.3267,  0.0006, -0.0157],
             [ 0.0241,  0.0001,  0.0191,  0.0074, -0.4548,  0.0027,  0.0702],
             [-0.0402, -0.0001, -0.0398, -0.0112,  0.7815, -0.0033, -0.0546]])
[26.4380595938345,22.96815598984732,-49.40621558368182]


# Decision Trees

In [51]:
from pyspark.ml.classification import DecisionTreeClassifier

In [52]:
target_variable_name = "y"
logistic_df = data.select(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'y'])
#exclude target variable and select all other feature vectors
features_list = logistic_df.columns
#features_list = char_vars #this option is used only for ChiSqselector
features_list.remove(target_variable_name)
# apply the function on our dataframe
binary_df = assemble_vectors(logistic_df, features_list, target_variable_name)

In [53]:
target_variable_name = "balance"
linear_df = data.select(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous'])
#exclude target variable and select all other feature vectors
features_list = linear_df.columns
#features_list = char_vars #this option is used only for ChiSqselector
features_list.remove(target_variable_name)
# apply the function on our dataframe
continuous_df = assemble_vectors(linear_df, features_list, target_variable_name)

In [54]:
from pyspark.ml.classification import DecisionTreeClassifier

clf = DecisionTreeClassifier(featuresCol='features', labelCol='y', impurity='gini')
clf_model = clf.fit(binary_df)
clf2 = DecisionTreeClassifier(featuresCol='features', labelCol='y', impurity='entropy')
clf_model2 = clf2.fit(binary_df)

In [55]:
print(clf_model.featureImportances)

(7,[0,2,3,4,5],[0.06244559784145926,0.0045805927987750735,0.7121851363817812,0.003176189598126988,0.2176124833798576])


In [56]:
print(clf_model2.featureImportances)

(7,[0,2,3,4,5],[0.016317599269546713,0.0011829483210483054,0.7201523674271109,0.00047276310126746204,0.2618743218810267])


In [57]:
from pyspark.ml.regression import DecisionTreeRegressor

reg = DecisionTreeRegressor(featuresCol='features', labelCol='balance', impurity='variance')
reg_model = reg.fit(continuous_df)

In [58]:
clf_model.toDebugString

'DecisionTreeClassificationModel: uid=DecisionTreeClassifier_67eefb7da282, depth=5, numNodes=35, numClasses=3, numFeatures=7\n  If (feature 3 <= 559.5)\n   If (feature 5 <= 9.5)\n    If (feature 0 <= 60.5)\n     Predict: 0.0\n    Else (feature 0 > 60.5)\n     If (feature 3 <= 125.5)\n      Predict: 0.0\n     Else (feature 3 > 125.5)\n      If (feature 3 <= 202.5)\n       Predict: 0.0\n      Else (feature 3 > 202.5)\n       Predict: 1.0\n   Else (feature 5 > 9.5)\n    If (feature 3 <= 166.5)\n     Predict: 0.0\n    Else (feature 3 > 166.5)\n     If (feature 5 <= 187.5)\n      If (feature 5 <= 92.5)\n       Predict: 1.0\n      Else (feature 5 > 92.5)\n       Predict: 0.0\n     Else (feature 5 > 187.5)\n      Predict: 0.0\n  Else (feature 3 > 559.5)\n   If (feature 3 <= 890.0)\n    If (feature 5 <= -0.5)\n     Predict: 0.0\n    Else (feature 5 > -0.5)\n     If (feature 5 <= 92.5)\n      Predict: 1.0\n     Else (feature 5 > 92.5)\n      If (feature 2 <= 21.5)\n       Predict: 0.0\n      El

## String to Json parser

In [59]:
def parse(lines):
    block = []
    while lines :

        if lines[0].startswith('If'):
            bl = ' '.join(lines.pop(0).split()[1:]).replace('(', '').replace(')', '')
            block.append({'id':bl, 'children':parse(lines)})


            if lines[0].startswith('Else'):
                be = ' '.join(lines.pop(0).split()[1:]).replace('(', '').replace(')', '')
                block.append({'id':be, 'children':parse(lines)})
        elif not lines[0].startswith(('If','Else')):
            block2 = lines.pop(0)
            block.append({'id':block2})
        else:
            break
    return block

def tree_json(tree):
    data = []
    for line in tree.splitlines() : 
        if line.strip():
            line = line.strip()
            data.append(line)
        else : break
        if not line : break
    res = []
    res.append({'id':'Root', 'children':parse(data[1:])})
    return res[0]

In [60]:
result = tree_json(clf_model.toDebugString)

In [62]:
from pyspark.sql.types import IntegerType
from pyspark.sql.types import StructField, StructType

cSchema = StructType([StructField("age", IntegerType())\
                      ,StructField("gender", IntegerType())\
                      ,StructField("y", IntegerType())])

test_list = [[30, 0, 1], 
             [25, 1, 0], 
             [45, 0, 0], 
             [57, 1, 1],
             [27, 0, 1], 
             [54, 1, 1], 
             [35, 1, 1]]


test_df = spark.createDataFrame(test_list, schema=cSchema)
test_df.show()

+---+------+---+
|age|gender|  y|
+---+------+---+
| 30|     0|  1|
| 25|     1|  0|
| 45|     0|  0|
| 57|     1|  1|
| 27|     0|  1|
| 54|     1|  1|
| 35|     1|  1|
+---+------+---+



In [63]:
test_df = assemble_vectors(test_df, ['age','gender'], 'y')
test_clf = DecisionTreeClassifier(featuresCol='features', labelCol='y')
test_clf_model = test_clf.fit(test_df)

In [64]:
clf_model.featureImportances

SparseVector(7, {0: 0.0624, 2: 0.0046, 3: 0.7122, 4: 0.0032, 5: 0.2176})

In [65]:
test_clf_model.toDebugString

'DecisionTreeClassificationModel: uid=DecisionTreeClassifier_dbab3d5101bb, depth=3, numNodes=7, numClasses=2, numFeatures=2\n  If (feature 0 <= 26.0)\n   Predict: 0.0\n  Else (feature 0 > 26.0)\n   If (feature 0 <= 40.0)\n    Predict: 1.0\n   Else (feature 0 > 40.0)\n    If (feature 0 <= 49.5)\n     Predict: 0.0\n    Else (feature 0 > 49.5)\n     Predict: 1.0\n'

# Random Forest

In [66]:
from pyspark.ml.classification import RandomForestClassifier

clf = RandomForestClassifier(featuresCol='features', labelCol='y')
clf_model = clf.fit(binary_df)
print(clf_model.featureImportances)
print(clf_model.toDebugString)

(7,[0,1,2,3,4,5,6],[0.09957407565411655,0.011195732980940013,0.021376143542810512,0.7144686698822783,0.00287945810458663,0.09092181392136983,0.05958410591389819])
RandomForestClassificationModel: uid=RandomForestClassifier_1c31cd5668eb, numTrees=20, numClasses=3, numFeatures=7
  Tree 0 (weight 1.0):
    If (feature 6 <= 0.5)
     If (feature 1 <= 864.5)
      If (feature 3 <= 562.5)
       If (feature 3 <= 380.5)
        Predict: 0.0
       Else (feature 3 > 380.5)
        If (feature 0 <= 60.5)
         Predict: 0.0
        Else (feature 0 > 60.5)
         Predict: 1.0
      Else (feature 3 > 562.5)
       If (feature 4 <= 13.5)
        Predict: 0.0
       Else (feature 4 > 13.5)
        If (feature 0 <= 42.5)
         Predict: 0.0
        Else (feature 0 > 42.5)
         Predict: 1.0
     Else (feature 1 > 864.5)
      If (feature 3 <= 478.5)
       Predict: 0.0
      Else (feature 3 > 478.5)
       If (feature 1 <= 4834.0)
        If (feature 3 <= 665.5)
         Predict: 0.0
      

In [67]:
from pyspark.ml.regression import RandomForestRegressor

reg = RandomForestRegressor(featuresCol='features', labelCol='balance')
reg_model = reg.fit(continuous_df)
print(reg_model.featureImportances)
print(reg_model.toDebugString)

(6,[0,1,2,3,4,5],[0.3861391097556759,0.23538795557721878,0.07857855517805655,0.044022337713233085,0.15373962056962298,0.10213242120619279])
RandomForestRegressionModel: uid=RandomForestRegressor_e7bbcf92c130, numTrees=20, numFeatures=6
  Tree 0 (weight 1.0):
    If (feature 0 <= 52.5)
     If (feature 1 <= 21.5)
      If (feature 4 <= 266.5)
       If (feature 1 <= 19.5)
        If (feature 5 <= 0.5)
         Predict: 1146.7013816746312
        Else (feature 5 > 0.5)
         Predict: 1712.9358033749083
       Else (feature 1 > 19.5)
        If (feature 4 <= 93.5)
         Predict: 1894.213847012842
        Else (feature 4 > 93.5)
         Predict: 2239.016055045872
      Else (feature 4 > 266.5)
       If (feature 5 <= 26.0)
        If (feature 4 <= 371.5)
         Predict: 839.6108786610879
        Else (feature 4 > 371.5)
         Predict: 2251.9912280701756
       Else (feature 5 > 26.0)
        If (feature 3 <= 3.5)
         Predict: 1995.3333333333333
        Else (feature 3 > 3.

# Gradient boosting

In [68]:
from pyspark.ml.classification import GBTClassifier

clf = GBTClassifier(featuresCol='features', labelCol='y')
clf_model = clf.fit(binary_df)
print(clf_model.featureImportances)
print(clf_model.toDebugString)

(7,[0,1,2,3,4,5,6],[0.10284731904525488,0.0940224178332045,0.1332119062065838,0.4671384156093845,0.043327106706113706,0.1425600056041739,0.016892828995284676])
GBTClassificationModel: uid = GBTClassifier_d5e9352b1060, numTrees=20, numClasses=2, numFeatures=7
  Tree 0 (weight 1.0):
    If (feature 3 <= 488.5)
     If (feature 5 <= 9.5)
      If (feature 0 <= 59.5)
       If (feature 3 <= 217.5)
        If (feature 0 <= 28.5)
         Predict: -0.8599088838268792
        Else (feature 0 > 28.5)
         Predict: -0.9637953828943354
       Else (feature 3 > 217.5)
        If (feature 0 <= 25.5)
         Predict: -0.5027624309392266
        Else (feature 0 > 25.5)
         Predict: -0.8417926003126629
      Else (feature 0 > 59.5)
       If (feature 3 <= 204.5)
        If (feature 3 <= 121.5)
         Predict: -0.9256965944272446
        Else (feature 3 > 121.5)
         Predict: -0.5974025974025974
       Else (feature 3 > 204.5)
        If (feature 3 <= 348.5)
         Predict: -0.309090

In [69]:
from pyspark.ml.regression import GBTRegressor

reg = GBTRegressor(featuresCol='features', labelCol='balance')
reg_model = reg.fit(continuous_df)
print(reg_model.featureImportances)
print(reg_model.toDebugString)

(6,[0,1,2,3,4,5],[0.36784974353071936,0.19151711910205355,0.2536685408510833,0.045300018528672326,0.09028176415352009,0.05138281383395142])
GBTRegressionModel: uid=GBTRegressor_df2b59877491, numTrees=20, numFeatures=6
  Tree 0 (weight 1.0):
    If (feature 0 <= 54.5)
     If (feature 0 <= 37.5)
      If (feature 1 <= 17.5)
       If (feature 1 <= 3.5)
        If (feature 4 <= 348.5)
         Predict: 1292.828346456693
        Else (feature 4 > 348.5)
         Predict: 2694.96
       Else (feature 1 > 3.5)
        If (feature 4 <= 24.0)
         Predict: 937.5397395002658
        Else (feature 4 > 24.0)
         Predict: 1198.0977011494253
      Else (feature 1 > 17.5)
       If (feature 1 <= 21.5)
        If (feature 0 <= 29.5)
         Predict: 917.4211287988422
        Else (feature 0 > 29.5)
         Predict: 1723.0375874125873
       Else (feature 1 > 21.5)
        If (feature 4 <= 2.5)
         Predict: 952.7784339457568
        Else (feature 4 > 2.5)
         Predict: 1535.129541

# Support vector machines

In [70]:
from pyspark.sql.functions import when

In [71]:
binary_df = binary_df.withColumn("y", when(binary_df["y"] == 2, 1).otherwise(binary_df["y"]))

In [72]:
from pyspark.ml.classification import LinearSVC

In [73]:
clf = LinearSVC(featuresCol='features', labelCol='y')
clf_model = clf.fit(binary_df)
print(clf_model.intercept, clf_model.coefficients)

-1.0000000574762387 [1.497446446917058e-10,-0.0,6.76696042699316e-10,1.488516520343328e-10,-4.993085808396494e-09,-0.0,8.620405268852088e-09]


In [74]:
from pyspark.ml.classification import LinearSVC
clf = LinearSVC(featuresCol='features', labelCol='y')
clf_model = clf.fit(binary_df)
print(clf_model.intercept, clf_model.coefficients)

-1.0000000574762387 [1.497446446917058e-10,-0.0,6.76696042699316e-10,1.488516520343328e-10,-4.993085808396494e-09,-0.0,8.620405268852088e-09]


In [75]:
import numpy as np
from pyspark.ml.classification import LogisticRegression

In [76]:
binary_clf = LogisticRegression(featuresCol='features', labelCol='y', family='binomial')

In [77]:
multinomial_clf = LogisticRegression(featuresCol='features', labelCol='y', family='multinomial')

In [78]:
target_variable_name = "y"
logistic_df = data.select(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'y'])
features_list = logistic_df.columns
features_list.remove(target_variable_name)

# assemble feature vectors using vector assembler
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=features_list, outputCol="features")
df = assembler.transform(logistic_df).select("features", target_variable_name)

# train multinomial logistic regression model
from pyspark.ml.classification import LogisticRegression
multinomial_clf = LogisticRegression(featuresCol='features', labelCol='y', family='multinomial')
multinomial_clf_model = multinomial_clf.fit(df)

# print model coefficients
import numpy as np
np.set_printoptions(precision=4, suppress=True)
print(multinomial_clf_model.coefficientMatrix)
print(multinomial_clf_model.interceptVector)


DenseMatrix([[ 0.0161,  0.    ,  0.0207,  0.0038, -0.3267,  0.0006, -0.0157],
             [ 0.0241,  0.0001,  0.0191,  0.0074, -0.4548,  0.0027,  0.0702],
             [-0.0402, -0.0001, -0.0398, -0.0112,  0.7815, -0.0033, -0.0546]])
[26.4380595938345,22.96815598984732,-49.40621558368182]


# Neural networks

In [80]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
#output_layer is set to 2 because of binary target
clf = MultilayerPerceptronClassifier(featuresCol='features', labelCol='y', layers=[4, 4, 2])
clf_model = clf.fit(binary_df)

# One vs rest classifier

In [81]:
target_variable_name = "education"
multiclass_df = data.select(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'job', 'education'])
features_list = multiclass_df.columns
#features_list = char_vars #this option is used only for ChiSqselector
features_list.remove(target_variable_name)
# apply the function on our dataframe
multiclass_df = assemble_vectors(multiclass_df, features_list, target_variable_name)

In [82]:
from pyspark.ml.classification import RandomForestClassifier, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# generate the train/test split.
(train, test) = multiclass_df.randomSplit([0.7, 0.3])
# instantiate the base classifier.
clf = RandomForestClassifier(featuresCol='features', labelCol='education')
# instantiate the One Vs Rest Classifier.
ovr = OneVsRest(classifier=clf, featuresCol='features', labelCol='education')
# train the multiclass model.
ovrModel = ovr.fit(train)
# score the model on test data.
predictions = ovrModel.transform(test)
# obtain evaluator.
evaluator = MulticlassClassificationEvaluator(metricName="accuracy", labelCol='education')
# compute the classification error on test data.
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.328868


# Naive Bayes classifier

In [83]:
target_variable_name = "y"
nonneg_df = data.select(['age', 'day', 'duration', 'campaign', 'previous', 'y'])
#exclude target variable and select all other feature vectors
features_list = nonneg_df.columns
#features_list = char_vars #this option is used only for ChiSqselector
features_list.remove(target_variable_name)
# apply the function on our dataframe
nonneg_df = assemble_vectors(nonneg_df, features_list, target_variable_name)

In [ ]:
from pyspark.ml.classification import NaiveBayes

#output_layer is set to 2 because of binary target
clf = NaiveBayes(featuresCol='features', labelCol='y')
clf_model = clf.fit(nonneg_df)